In [17]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN, KMeans
from scipy.spatial.distance import mahalanobis
from shapely.geometry import Polygon
from scipy.stats import median_abs_deviation, zscore
from ensemble_boxes import weighted_boxes_fusion
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score

# ----- Step 1: IOU Calculation -----

def iou(box1, box2):
    poly1 = Polygon([(box1[0], box1[1]), (box1[2], box1[1]), (box1[2], box1[3]), (box1[0], box1[3])])
    poly2 = Polygon([(box2[0], box2[1]), (box2[2], box2[1]), (box2[2], box2[3]), (box2[0], box2[3])])
    intersection = poly1.intersection(poly2).area
    union = poly1.union(poly2).area
    return intersection / union

# ----- Step 2: Clustering Bounding Boxes -----

def dbscan_clustering(boxes, eps=0.5, min_samples=2):
    iou_matrix = np.array([[iou(box1, box2) for box2 in boxes] for box1 in boxes])
    clustering = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed').fit(1 - iou_matrix)
    return clustering.labels_

# ----- Step 3: Mahalanobis and Tukey Fences Outlier Detection -----

def mahalanobis_outliers(boxes, median_box, reg=1e-6):
    cov_matrix = np.cov(np.array(boxes).T)
    cov_matrix += np.eye(cov_matrix.shape[0]) * reg  # Add regularization
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    distances = [mahalanobis(box, median_box, inv_cov_matrix) for box in boxes]
    return distances

def tukey_fences_outliers(distances, k=3):

    q1, q3 = np.percentile(distances, [25, 75])
    iqr = q3 - q1
    lower_fence = q1 - k * iqr
    upper_fence = q3 + k * iqr
    return [i for i, d in enumerate(distances) if d < lower_fence or d > upper_fence]

# ----- Step 4: Z-Score Outlier Detection -----

def z_score_outliers(boxes, threshold=3):
    z_scores = np.abs(zscore(boxes, axis=0))
    # If any coordinate of a box has a Z-score greater than threshold, mark as an outlier
    outliers = np.where(np.any(z_scores > threshold, axis=1))[0]
    return outliers

# ----- Step 5: Feature Extraction using SIFT, ORB, and HOG -----

def extract_sift_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

def extract_orb_features(image):
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    return keypoints, descriptors

def extract_hog_features(image):
    hog = cv2.HOGDescriptor()
    return hog.compute(image)

def validate_box_with_features(image, box, sift_thresh=10, orb_thresh=10, hog_thresh=10):
    x1, y1, x2, y2 = map(int, box)
    cropped = image[y1:y2, x1:x2]
    print(cropped)
    # Extract features from the cropped image
    sift_kp, _ = extract_sift_features(cropped)
    orb_kp, _ = extract_orb_features(cropped)
    hog_features = extract_hog_features(cropped)

    # Check if the box is valid based on the extracted features
    return (len(sift_kp) > sift_thresh and 
            len(orb_kp) > orb_thresh and 
            len(hog_features) > hog_thresh)

# ----- Step 6: Bayesian Inference for User Confidence -----

def bayesian_update(prior, likelihood):
    posterior = prior * likelihood / ((prior * likelihood) + ((1 - prior) * (1 - likelihood)))
    return posterior

def compute_confidence(user_boxes, prior_confidences, likelihoods):
    confidences = [bayesian_update(prior, likelihood) for prior, likelihood in zip(prior_confidences, likelihoods)]
    return confidences

# ----- Step 7: Aggregating Bounding Boxes via K-means -----

def kmeans_bounding_box_aggregation(boxes, n_clusters=1):
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(boxes)
    consensus_box = kmeans.cluster_centers_[0]
    return consensus_box

# ----- Step 8: Weighted Box Fusion (WBF) -----

def apply_wbf(boxes, scores, labels, iou_thresh=0.5):
    # Applying Weighted Box Fusion
    boxes = [[box[0] / 640, box[1] / 480, box[2] / 640, box[3] / 480] for box in boxes]  # Normalize
    fused_boxes, fused_scores, _ = weighted_boxes_fusion([boxes], [scores], [labels], iou_thr=iou_thresh, skip_box_thr=0.0)
    fused_boxes = [[box[0] * 640, box[1] * 480, box[2] * 640, box[3] * 480] for box in fused_boxes]  # De-normalize
    return fused_boxes[0]  # Return the final fused box

# ----- Step 9: Geometric Median for Robust Consensus -----

def geometric_median(points, eps=1e-5):
    median = np.median(points, axis=0)
    while True:
        distances = np.linalg.norm(points - median, axis=1)
        new_median = np.average(points, weights=1 / np.maximum(distances, eps), axis=0)
        if np.linalg.norm(new_median - median) < eps:
            break
        median = new_median
    return median

# ----- Step 10: Graph-Based Outlier Detection -----

def graph_based_outlier_detection(boxes, threshold=0.5):
    from sklearn.neighbors import NearestNeighbors
    neigh = NearestNeighbors(n_neighbors=2)
    neigh.fit(boxes)
    distances, indices = neigh.kneighbors(boxes)
    outliers = np.where(distances[:, 1] > threshold)[0]
    return outliers

# ----- Step 11: Calibration with Platt Scaling and Isotonic Regression -----

def calibrate_with_platt(likelihoods, labels):
    platt_model = LogisticRegression()
    platt_model.fit(likelihoods.reshape(-1, 1), labels)
    calibrated_scores = platt_model.predict_proba(likelihoods.reshape(-1, 1))[:, 1]
    return calibrated_scores

def calibrate_with_isotonic(likelihoods, labels):
    iso_model = IsotonicRegression(out_of_bounds="clip")
    calibrated_scores = iso_model.fit_transform(likelihoods, labels)
    return calibrated_scores

# ----- Main Algorithm -----
def validate_bounding_boxes(image, user_boxes, prior_confidences, user_labels):
    # Step 1: IOU-based Clustering to Remove Outliers
    labels = dbscan_clustering(user_boxes)
    print(f"DBSCAN Clustering Labels: {labels}")
    
    clustered_boxes = [user_boxes[i] for i in range(len(labels)) if labels[i] != -1]
    print(f"Clustered Bounding Boxes (after removing outliers): {clustered_boxes}")
    
    # Step 2: Compute Geometric Median as Robust Consensus
    geometric_consensus_box = geometric_median(clustered_boxes)
    print(f"Geometric Consensus Box: {geometric_consensus_box}")
    
    # Step 3: Detect Outliers with Mahalanobis Distance and Tukey Fences
    mahalanobis_distances = mahalanobis_outliers(clustered_boxes, geometric_consensus_box)
    print(f"Mahalanobis Distances: {mahalanobis_distances}")
    
    tukey_outliers_idx = tukey_fences_outliers(mahalanobis_distances, k=10)
    print(f"Tukey Outliers Indices: {tukey_outliers_idx}")
    
    valid_boxes = [clustered_boxes[i] for i in range(len(clustered_boxes)) if i not in tukey_outliers_idx]
    print(f"Valid Bounding Boxes (after removing Tukey outliers): {valid_boxes}")
    
    # Step 4: Z-Score Outlier Detection
    z_outliers_idx = z_score_outliers(valid_boxes, threshold=3)
    print(f"Z-Score Outliers Indices: {z_outliers_idx}")
    
    valid_boxes = [valid_boxes[i] for i in range(len(valid_boxes)) if i not in z_outliers_idx]
    print(f"Valid Bounding Boxes (after removing Z-Score outliers): {valid_boxes}")
    
    # Step 5: Validate Boxes using SIFT, ORB, and HOG Feature Extraction
    likelihoods = [validate_box_with_features(image, box) for box in valid_boxes]
    print(f"Likelihood Scores from Feature Validation: {likelihoods}")
    
    # Step 6: Update User Confidence Scores with Bayesian Inference
    final_confidences = compute_confidence(valid_boxes, prior_confidences, likelihoods)
    print(f"Final Confidences after Bayesian Inference: {final_confidences}")
    
    # Step 7: Apply Weighted Box Fusion (WBF) to get the final box
    final_box_wbf = apply_wbf(valid_boxes, final_confidences, user_labels)
    print(f"Final Box from Weighted Box Fusion (WBF): {final_box_wbf}")
    
    final_box_kmeans = kmeans_bounding_box_aggregation(valid_boxes, 1)
    print(f"Final Box from K-Means Aggregation: {final_box_kmeans}")
    
    final_box = kmeans_bounding_box_aggregation([final_box_wbf, final_box_kmeans], 1)
    print(f"Final Bounding Box (combined WBF and K-Means): {final_box}")
    
    # Step 8: Calibrate the confidence scores using Platt Scaling and Isotonic Regression
    platt_calibrated_scores = calibrate_with_platt(np.array(likelihoods), user_labels)
    print(f"Platt Calibrated Confidence Scores: {platt_calibrated_scores}")
    
    isotonic_calibrated_scores = calibrate_with_isotonic(np.array(likelihoods), user_labels)
    print(f"Isotonic Calibrated Confidence Scores: {isotonic_calibrated_scores}")

    return final_box, platt_calibrated_scores, isotonic_calibrated_scores


    

In [7]:
!pip3 install opencv-python scikit-learn scipy shapely ensemble-boxes


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.3 MB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 28.8 MB 4.0 MB/s eta 0:00:01     |████████████████████████████    | 25.3 MB 145 kB/s eta 0:00:25
     |████████████████████████████████| 505 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 345 kB 5.4 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import json

# Path to your JSON file
file_path = 'modified_annotations_with_outliers.json'

# Read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Extract the images list
images_list = data['images']

images_list

[{'id': 0,
  'license': 1,
  'file_name': 'image24_jpeg_jpg.rf.0051bfdaa975eb968a84d54bfaaf131b.jpg',
  'height': 640,
  'width': 640,
  'date_captured': '2022-12-08T17:01:50+00:00'},
 {'id': 1,
  'license': 1,
  'file_name': 'xemay1530_jpg.rf.0056c921910b81e8645fc67f6128b140.jpg',
  'height': 640,
  'width': 640,
  'date_captured': '2022-12-08T17:01:50+00:00'},
 {'id': 2,
  'license': 1,
  'file_name': 'Cars192_png_jpg.rf.005c0aaba03c0cf6da2f72c615f5d852.jpg',
  'height': 640,
  'width': 640,
  'date_captured': '2022-12-08T17:01:50+00:00'},
 {'id': 3,
  'license': 1,
  'file_name': 'CarLongPlateGen1346_jpg.rf.0070c476472e8da57fccc4caec5c58f3.jpg',
  'height': 640,
  'width': 640,
  'date_captured': '2022-12-08T17:01:50+00:00'},
 {'id': 4,
  'license': 1,
  'file_name': 'pic_102_jpg.rf.01431b9af693f31cbbe7fcbc5b38a759.jpg',
  'height': 640,
  'width': 640,
  'date_captured': '2022-12-08T17:01:50+00:00'},
 {'id': 5,
  'license': 1,
  'file_name': 'xemay274_jpg.rf.016a456f6fd379eb4fd4092

In [8]:
annotated_list = data['annotations']
annotated_list

[{'id': 0,
  'image_id': 0,
  'category_id': 1,
  'area': 39900,
  'segmentation': [],
  'iscrowd': 0,
  'bbox1': [87.97441875620854,
   226.46131399687232,
   260.7791205327014,
   163.66764778250894],
  'bbox2': [89.76073115192322,
   226.31659768109546,
   241.98507635788323,
   166.46541305694547],
  'bbox3': [0, 0, 474.8626980682823, 209.8934858843977],
  'bbox4': [0, 523.7010242778049, 168.10805806067336, 152.59244399879952],
  'area1': 42681.10524837862,
  'area2': 40282.14568953152,
  'area3': 99670.58701402202,
  'area4': 25652.01943537024},
 {'id': 1,
  'image_id': 1,
  'category_id': 1,
  'area': 20054,
  'segmentation': [],
  'iscrowd': 0,
  'bbox1': [160.40742675290505,
   301.02619924127686,
   144.0332454992498,
   141.26188950072367],
  'bbox2': [173.93234809611738,
   294.6338083489277,
   144.72511799513177,
   141.6771859224584],
  'bbox3': [173.76877181768634,
   301.1558460120948,
   125.02975676471227,
   152.65647522262609],
  'bbox4': [166.46868941412595,
   289

In [9]:
dict = {}

for img in images_list:
    dict[img['file_name']] = [[box['bbox1'], box['bbox2'], box['bbox3'], box['bbox4']] for box in annotated_list if box['image_id'] == img['id']]

dict

{'image24_jpeg_jpg.rf.0051bfdaa975eb968a84d54bfaaf131b.jpg': [[[87.97441875620854,
    226.46131399687232,
    260.7791205327014,
    163.66764778250894],
   [89.76073115192322,
    226.31659768109546,
    241.98507635788323,
    166.46541305694547],
   [0, 0, 474.8626980682823, 209.8934858843977],
   [0, 523.7010242778049, 168.10805806067336, 152.59244399879952]]],
 'xemay1530_jpg.rf.0056c921910b81e8645fc67f6128b140.jpg': [[[160.40742675290505,
    301.02619924127686,
    144.0332454992498,
    141.26188950072367],
   [173.93234809611738,
    294.6338083489277,
    144.72511799513177,
    141.6771859224584],
   [173.76877181768634,
    301.1558460120948,
    125.02975676471227,
    152.65647522262609],
   [166.46868941412595,
    289.87428353591423,
    134.99547490200976,
    161.5310387236131]]],
 'Cars192_png_jpg.rf.005c0aaba03c0cf6da2f72c615f5d852.jpg': [[[192.76844842295395,
    418.98337512671594,
    198.86816628755378,
    68.00802215826138],
   [202.42182974086845,
    419.12

In [18]:
for img, boxes in dict.items():
    image = cv2.imread(f'test/{img}')
    if image is None:
        print(f"Failed to load image: {img}")
        
    else:
        
        print(f"Image loaded successfully: {img}")
    print(validate_bounding_boxes(image, boxes[0], [0.5, 0.5, 0.5, 0.5], [1, 2, 3, 4]))
    # print(extract_sift_features(image=image))
    

Image loaded successfully: image24_jpeg_jpg.rf.0051bfdaa975eb968a84d54bfaaf131b.jpg
DBSCAN Clustering Labels: [ 0  0 -1 -1]
Clustered Bounding Boxes (after removing outliers): [[87.97441875620854, 226.46131399687232, 260.7791205327014, 163.66764778250894], [89.76073115192322, 226.31659768109546, 241.98507635788323, 166.46541305694547]]
Geometric Consensus Box: [ 88.86757495 226.38895584 251.38209845 165.06653042]
Mahalanobis Distances: [np.float64(0.7071067792613778), np.float64(0.7071067792148524)]
Tukey Outliers Indices: []
Valid Bounding Boxes (after removing Tukey outliers): [[87.97441875620854, 226.46131399687232, 260.7791205327014, 163.66764778250894], [89.76073115192322, 226.31659768109546, 241.98507635788323, 166.46541305694547]]
Z-Score Outliers Indices: []
Valid Bounding Boxes (after removing Z-Score outliers): [[87.97441875620854, 226.46131399687232, 260.7791205327014, 163.66764778250894], [89.76073115192322, 226.31659768109546, 241.98507635788323, 166.46541305694547]]
[]


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/features2d/src/sift.dispatch.cpp:512: error: (-5:Bad argument) image is empty or has incorrect depth (!=CV_8U) in function 'detectAndCompute'


In [20]:
import numpy as np

# Function to calculate IoU between two bounding boxes
def calculate_iou(boxA, boxB):
    # Coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    
    # Compute the area of intersection
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    # Compute the area of both bounding boxes
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    
    # Compute the IoU
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

# Function to calculate weighted average of bounding boxes
def weighted_average_bbox(boxes, weights):
    total_weight = sum(weights)
    x_avg = sum([w * box[0] for box, w in zip(boxes, weights)]) / total_weight
    y_avg = sum([w * box[1] for box, w in zip(boxes, weights)]) / total_weight
    width_avg = sum([w * box[2] for box, w in zip(boxes, weights)]) / total_weight
    height_avg = sum([w * box[3] for box, w in zip(boxes, weights)]) / total_weight
    return [x_avg, y_avg, width_avg, height_avg]

# Function to find clusters of bounding boxes using IoU threshold
def cluster_bboxes(bboxes, iou_threshold=0.5):
    clusters = []
    for box in bboxes:
        added = False
        for cluster in clusters:
            if calculate_iou(cluster[0], box) > iou_threshold:
                cluster.append(box)
                added = True
                break
        if not added:
            clusters.append([box])
    return clusters

# Example bounding boxes from users: [x, y, width, height]
bboxes = [
    [50, 50, 100, 100],  # User 1
    [52, 48, 102, 98],   # User 2
    [200, 200, 100, 100] # User 3 (outlier)
]

# Example user weights (based on their performance)
weights = [0.8, 0.9, 0.5, 0.8]

# Step 1: Cluster bounding boxes based on IoU
clusters = cluster_bboxes(bboxes, iou_threshold=0.5)

# Step 2: Calculate consensus bounding box for each cluster
consensus_bboxes = []
for cluster in clusters:
    # Get the weights for the users in the current cluster
    cluster_weights = [weights[bboxes.index(box)] for box in cluster]
    # Calculate the weighted average bounding box for this cluster
    consensus_box = weighted_average_bbox(cluster, cluster_weights)
    consensus_bboxes.append(consensus_box)

# Output final consensus bounding boxes

print("Consensus Bounding Boxes:", consensus_bboxes)

for _, bboxes in dict.items():
    cluster_weights = [weights[bboxes.index(box)] for box in cluster]
    # Calculate the weighted average bounding box for this cluster
    consensus_box = weighted_average_bbox(cluster, cluster_weights)
    consensus_bboxes.append(consensus_box)
    # print(extract_sift_features(image=image))


Consensus Bounding Boxes: [[51.05882352941177, 48.94117647058823, 101.05882352941175, 98.94117647058822], [200.0, 200.0, 100.0, 100.0]]


ValueError: [200, 200, 100, 100] is not in list